In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from utils import metricsCal

In [29]:
#硬投票
type_name = ["FullTranscript","matureRNA"]
cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]
feature_name = ["emb","PSNP","PCP","DBPF"]
Param_HardVoting = np.zeros(60*6).reshape(60,6)

row_name = []
rows = -1
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        true = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,1])
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold0"+".csv").iloc[:,10]
        

        for i in range(5):
            rows+=1
            row_name.append([type_name[typei]+"_"+cell_name[celli]+"_Kfold_"+str(i)])
            #testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)

            #_,_,_,_,sen,spe, acc,mcc, _ = metricsCal.get_test_metrics(np.mean(testData,axis=1),testLabel.reshape(-1),th)
            sen,spe,acc,mcc,_ = para4(test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],testLabel)
            auc = roc4(test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],testLabel)
            print(auc)

            Param_HardVoting[typei*30+celli*5+i][0] = sen
            Param_HardVoting[typei*30+celli*5+i][1] = spe
            Param_HardVoting[typei*30+celli*5+i][2] = acc
            Param_HardVoting[typei*30+celli*5+i][3] = mcc
            Param_HardVoting[typei*30+celli*5+i][4] = auc
            Param_HardVoting[typei*30+celli*5+i][5] = 0.5

0.8369134130221673
0.8320722548443741
0.8333106952308955
0.8334442049121568
0.8334637208911827
0.8195708929038975
0.8217861444540192
0.8163989876604295
0.8218622915748132
0.8268710593250349
0.8013748437796614
0.7992921291149182
0.7981287822963199
0.8015407471520528
0.7979710445487671
0.8588807664670353
0.8603319375493447
0.858661297267235
0.8612959440027316
0.8623714487011819
0.7590117655509753
0.7643458626074344
0.7636644701347035
0.7608975933932123
0.7606352028539645
0.7830693689171951
0.7855467499082633
0.7868930727665365
0.7825057273506834
0.7817145687614953
0.8086577077551619
0.8039673149021365
0.803950820513501
0.812089531707563
0.8068379577561271
0.78602514038337
0.792875059063528
0.7913277248560122
0.7937064015112698
0.7902719062050179
0.7356581278045412
0.731209203771343
0.7307757818124436
0.732212265602074
0.7393404776147686
0.7880184011188165
0.8032064452936248
0.7885167817466301
0.7945336147727463
0.7901385329812661
0.7290393337937271
0.732979381399296
0.7340476031724311
0.

In [30]:
np.mean(Param_HardVoting[:,4]),np.mean(Param_HardVoting[:,3])

(0.7866591873582823, 0.45068027420584494)

In [27]:
header_name = ["name",'Sen', 'Spe', 'Acc', 'mcc', 'AUC','th']
pd.DataFrame(np.append(np.array(row_name).reshape(-1,1),Param_HardVoting,axis=1)).to_csv("Table/Hard_Voting_result_iter_0.3_0.7.csv",header=header_name,index=0)

In [28]:
def para4(x1, x2, x3, x4, label, th=0.5):
    num_pos = np.int(np.sum(label))
    num_neg = len(label)-num_pos
    
    max1 = float(max(x1))
    min1 = float(min(x1))
    
    max2 = float(max(x2))
    min2 = float(min(x2))
    
    max3 = float(max(x3))
    min3 = float(min(x3))
    
    max4 = float(max(x4))
    min4 = float(min(x4))

    max6 = max(max1,max2,max3,max4)
    min6 = min(min1,min2,min3,min4)
    
    pred =[]
    for j in range(len(x1)):
        temp1l = 0
        if x1[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x2[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x3[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x4[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        
        if temp1l>=0:
            pred.append(1)
        elif temp1l<0:
            pred.append(0)
#         elif np.mean([x1[j],x2[j],x3[j],x4[j]])>th:
#             pred.append(1)
#         else:
#             pred.append(0)
    TN, FN, FP, TP, Sen, Spe, Acc, mcc, AUC = metricsCal.get_test_metrics(np.array(pred),label,th)
    return Sen, Spe, Acc, mcc, AUC

In [20]:
def roc4(x1, x2, x3, x4, label, iter = 10, low = 0.30 , high = 0.70):
    num_pos = np.int(np.sum(label))
    num_neg = len(label)-num_pos
    
    max1 = float(max(x1))
    min1 = float(min(x1))
    
    max2 = float(max(x2))
    min2 = float(min(x2))
    
    max3 = float(max(x3))
    min3 = float(min(x3))
    
    max4 = float(max(x4))
    min4 = float(min(x4))
    
    max6 = max(max1,max2,max3,max4)
    min6 = min(min1,min2,min3,min4)
    
    TPR = []
    FPR = []
    
    for i in range(iter+1):
        if i == 0 or (i/iter > low and i/iter < high) or i == iter:
            bool_ = min6+i*(max6-min6)/iter
            tp = 0
            tn = 0
            for j in range(len(x1)):
                temp1l = 0
                if x1[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x2[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x3[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x4[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                
                if temp1l>0 and label[j]==1:
                    tp += 1
                elif temp1l<0 and label[j]==0:
                    tn += 1
                elif np.mean([x1[j],x2[j],x3[j],x4[j]])>bool_ and label[j]==1:
                    tp += 1
                elif np.mean([x1[j],x2[j],x3[j],x4[j]])<bool_ and label[j]==0:
                    tn += 1
            tpr_ = tp/num_pos
            fpr_ = 1- tn/num_neg
            TPR.append(tpr_)
            FPR.append(fpr_)
    area = 0
    for i in range(len(TPR)-1):
        temp = (-FPR[i+1]+FPR[i])*(TPR[i]+TPR[i+1])/2
        area += temp 
    return area

In [6]:
def roc5(x1, x2, x3, x4, x5, label, iter = 10, low = 0.01 , high = 0.99):
    num_pos = np.int(np.sum(label))
    num_neg = len(label)-num_pos
    
    max1 = float(max(x1))
    min1 = float(min(x1))
    
    max2 = float(max(x2))
    min2 = float(min(x2))
    
    max3 = float(max(x3))
    min3 = float(min(x3))
    
    max4 = float(max(x4))
    min4 = float(min(x4))
    
    max5 = float(max(x5))
    min5 = float(min(x5))
    
    max6 = max(max1,max2,max3,max4,max5)
    min6 = min(min1,min2,min3,min4,min5)
    
    TPR = []
    FPR = []
    
    for i in range(iter+1):
        if i == 0 or (i/iter > low and i/iter < high) or i == iter:
            bool_ = min6+i*(max6-min6)/iter
            tp = 0
            tn = 0
            for j in range(len(x1)):
                temp1l = 0
                if x1[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x2[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x3[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x4[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x5[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if temp1l>0 and label[j]==1:
                    tp += 1
                elif temp1l<0 and label[j]==0:
                    tn += 1
            tpr_ = tp/num_pos
            fpr_ = 1- tn/num_neg
            TPR.append(tpr_)
            FPR.append(fpr_)
    area = 0
    for i in range(len(TPR)-1):
        temp = (-FPR[i+1]+FPR[i])*(TPR[i]+TPR[i+1])/2
        area += temp 
    return area

In [5]:
def para5(x1, x2, x3, x4, x5, label, th=0.5):
    num_pos = np.int(np.sum(label))
    num_neg = len(label)-num_pos
    
    max1 = float(max(x1))
    min1 = float(min(x1))
    
    max2 = float(max(x2))
    min2 = float(min(x2))
    
    max3 = float(max(x3))
    min3 = float(min(x3))
    
    max4 = float(max(x4))
    min4 = float(min(x4))
    
    max5 = float(max(x5))
    min5 = float(min(x5))
    
    max6 = max(max1,max2,max3,max4,max5)
    min6 = min(min1,min2,min3,min4,min5)
    
    pred =[]
    for j in range(len(x1)):
        temp1l = 0
        if x1[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x2[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x3[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x4[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x5[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        
        if temp1l>0:
            pred.append(1)
        elif temp1l<0:
            pred.append(0)
    TN, FN, FP, TP, Sen, Spe, Acc, mcc, AUC = metricsCal.get_test_metrics(np.array(pred),label,th)
    return Sen, Spe, Acc, mcc

In [10]:
def roc4_new(x1, x2, x3, x4, label, iter = 10, low = 0.01 , high = 0.99):
    num_pos = np.int(np.sum(label))
    num_neg = len(label)-num_pos
    
    max1 = float(max(x1))
    min1 = float(min(x1))
    
    max2 = float(max(x2))
    min2 = float(min(x2))
    
    max3 = float(max(x3))
    min3 = float(min(x3))
    
    max4 = float(max(x4))
    min4 = float(min(x4))
    
    max6 = max(max1,max2,max3,max4)
    min6 = min(min1,min2,min3,min4)
    
    TPR = []
    FPR = []
    
    for i in range(iter+1):
        if i == 0 or (i/iter > low and i/iter < high) or i == iter:
            bool_ = min6+i*(max6-min6)/iter
            tp = 0
            tn = 0
            for j in range(len(x1)):
                temp1l = 0
                if x1[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x2[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x3[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                if x4[j]>bool_:
                    temp1l+=1
                else:
                    temp1l-=1
                
                if temp1l>0 and label[j]==1:
                    tp += 1
                elif temp1l<0 and label[j]==0:
                    tn += 1
#                 elif np.mean([x1[j],x2[j],x3[j],x4[j]])>bool_ and label[j]==1:
#                     tp += 1
#                 elif np.mean([x1[j],x2[j],x3[j],x4[j]])<bool_ and label[j]==0:
#                     tn += 1
            tpr_ = tp/num_pos
            fpr_ = 1- tn/num_neg
            TPR.append(tpr_)
            FPR.append(fpr_)
    area = 0
    for i in range(len(TPR)-1):
        temp = (-FPR[i+1]+FPR[i])*(TPR[i]+TPR[i+1])/2
        area += temp 
    return area

In [16]:
def para4_new(x1, x2, x3, x4, label, th=0.5):
    num_pos = np.int(np.sum(label))
    num_neg = len(label)-num_pos
    
    max1 = float(max(x1))
    min1 = float(min(x1))
    
    max2 = float(max(x2))
    min2 = float(min(x2))
    
    max3 = float(max(x3))
    min3 = float(min(x3))
    
    max4 = float(max(x4))
    min4 = float(min(x4))

    max6 = max(max1,max2,max3,max4)
    min6 = min(min1,min2,min3,min4)
    
    pred =[]
    for j in range(len(x1)):
        temp1l = 0
        if x1[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x2[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x3[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        if x4[j]>th:
            temp1l+=1
        else:
            temp1l-=1
        
        if temp1l>=0:
            pred.append(1)
        elif temp1l<0:
            pred.append(0)
#         elif np.mean([x1[j],x2[j],x3[j],x4[j]])>th:
#             pred.append(1)
#         else:
#             pred.append(0)
    TN, FN, FP, TP, Sen, Spe, Acc, mcc, AUC = metricsCal.get_test_metrics(np.array(pred),label,th)
    return Sen, Spe, Acc, mcc, AUC

In [18]:
#硬投票
type_name = ["FullTranscript","matureRNA"]
cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]
feature_name = ["emb","PSNP","PCP","DBPF"]
Param_HardVoting_1 = np.zeros(60*6).reshape(60,6)

row_name = []
rows = -1
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        true = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"val.csv").iloc[:,1])
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        
        test_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold0"+".csv").iloc[:,10]
        

        for i in range(5):
            rows+=1
            row_name.append([type_name[typei]+"_"+cell_name[celli]+"_Kfold_"+str(i)])
            #testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)

            #_,_,_,_,sen,spe, acc,mcc, _ = metricsCal.get_test_metrics(np.mean(testData,axis=1),testLabel.reshape(-1),th)
            sen,spe,acc,mcc,_ = para4_new(test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],testLabel)
            auc = roc4_new(test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],testLabel)
            print(auc)

            Param_HardVoting_1[typei*30+celli*5+i][0] = sen
            Param_HardVoting_1[typei*30+celli*5+i][1] = spe
            Param_HardVoting_1[typei*30+celli*5+i][2] = acc
            Param_HardVoting_1[typei*30+celli*5+i][3] = mcc
            Param_HardVoting_1[typei*30+celli*5+i][4] = auc
            Param_HardVoting_1[typei*30+celli*5+i][5] = 0.5

0.7668453501731494
0.7677192614172608
0.7681798279194795
0.7677250006063258
0.7679603706327255
0.7549901250228366
0.7533949473310785
0.7492346456303675
0.758986265823925
0.7542275556573168
0.7287416981722478
0.7378753934965119
0.724257416476065
0.7272524541341511
0.736430194343909
0.7859002328450037
0.7852964085687886
0.7899390998379002
0.7988061895854381
0.7965988631124786
0.692937570159446
0.7062114655696307
0.7037374647137714
0.7003354734684235
0.6961166996416971
0.7069206538361823
0.7187132146912127
0.7122794305323704
0.7095155922686281
0.7075807104523248
0.7197561771658411
0.7072599567263073
0.7110147496642347
0.7251666358757576
0.7206118235762958
0.6897714293362356
0.6991105948968233
0.6983792486898991
0.7015231429515592
0.697714732060845
0.6404090110316806
0.6220142509070029
0.6216560093262783
0.6346451382523515
0.62874821473968
0.6946941145664088
0.7047974928109759
0.6881296206265645
0.706580839401006
0.6953292804746387
0.6326183516232016
0.6478586565915643
0.6487080736534575
0

In [17]:
np.mean(Param_HardVoting_1[:,4]),np.mean(Param_HardVoting_1[:,3])

(0.7047008324430413, 0.45343689336412357)

In [19]:
np.mean(Param_HardVoting_1[:,4]),np.mean(Param_HardVoting_1[:,3])

(0.7047008324430413, 0.45068027420584494)

In [36]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [43]:
#Logistic回归模型,训练新数据测试集测试新数据
header_name = ["name","Sen","Spe","Acc","Mcc","Auc","th"]
Param = np.zeros(60*6).reshape(60,6)
row_name = []
rows = -1
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]

        
        newTestData = np.array([])
        newTestLabel = np.array([])
        for i in range(5):
            rows+=1
            row_name.append([type_name[typei]+"_"+cell_name[celli]+"_KFold_"+str(i)])

            testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i]]).T)

            newTestData_one = np.array([])
            newTestLabel_one = np.array([])

            kf = KFold(5,True,10)
            Sen = 0
            Spe = 0
            Acc = 0
            Mcc = 0
            Auc = 0
            Th = 0
            for j,[train_index, test_index] in enumerate(kf.split(testData)):

                X_train = testData[train_index]
                X_test = testData[test_index]
                Y_train = testLabel[train_index]
                Y_test = testLabel[test_index]

                model = LogisticRegression()
                clf =model.fit(X_train, Y_train)
                th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(X_train).reshape(-1),Y_train.reshape(-1))
                _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics(clf.decision_function(X_test).reshape(-1),Y_test.reshape(-1),th)
                newTestData_one = np.append(newTestData_one,clf.decision_function(X_test).reshape(-1),axis = 0)
                newTestLabel_one = np.append(newTestLabel_one,Y_test.reshape(-1),axis = 0)

                Sen+=sen
                Spe+=spe
                Acc+=acc
                Mcc+=mcc
                Auc+=auc
                Th+=th
                print(auc)
            if i == 0:
                newTestLabel = np.append(newTestLabel,newTestLabel_one,axis=0)
                newTestData = np.append(newTestData,newTestData_one,axis = 0)
            else:
                newTestData = np.append(newTestData.reshape(-1,i),newTestData_one.reshape(-1,1),axis = 1)

            Param[rows][0] = Sen/5
            Param[rows][1] = Spe/5
            Param[rows][2] = Acc/5
            Param[rows][3] = Mcc/5
            Param[rows][4] = Auc/5
            Param[rows][5] = Th/5
        pd.DataFrame(np.append(newTestLabel.reshape(-1,1),newTestData,axis=1)).to_csv("Result/Stacking_Fusion/"+type_name[typei]+"_"+cell_name[celli]+"testFold.csv",header = ["true","KFold_0","KFold_1","KFold_2","KFold_3","KFold_4"])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8620221452695482
0.8620748000366617
0.8543591885540518
0.8667032698251429
0.8651290942976424


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8551405241332137
0.859253442844069
0.8576095528057407
0.8624884332308786
0.8668495476893225


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


...

0.7458789138782875
0.7578169990733659
0.7532124461224317
0.7393477490251684
0.7283135957616816


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7396090028949285
0.7544040000746284
0.7459481457162825
0.7353579357823841
0.7227019901317324


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7364021169610739
0.7555909003277423
0.7501053923580281
0.7291330062128024
0.7263219094818072


In [44]:
pd.DataFrame(np.append(np.array(row_name).reshape(-1,1),Param,axis=1)).to_csv("Table/Stacking_Result_no_Blastn.csv",header=header_name,index=0)

In [48]:
np.mean(Param[:,-2])

0.8130155846217332

In [49]:
#Logistic回归模型,训练新数据测试集测试新数据,带Blastn的
header_name = ["name","Sen","Spe","Acc","Mcc","Auc","th"]
Param = np.zeros(60*6).reshape(60,6)
row_name = []
rows = -1
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        
        testLabel = np.array(pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,1])
        test_1 = pd.read_csv("Result/"+feature_name[0]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_2 = pd.read_csv("Result/"+feature_name[1]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_3 = pd.read_csv("Result/"+feature_name[2]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_4 = pd.read_csv("Result/"+feature_name[3]+"/"+type_name[typei]+"_"+cell_name[celli]+"test_result.csv").iloc[:,2:7]
        test_Blastn = pd.read_csv("Stacking_result/all_test_KFold/test_"+type_name[typei]+"_"+cell_name[celli]+"_Kfold0"+".csv").iloc[:,10]
        

        
        newTestData = np.array([])
        newTestLabel = np.array([])
        for i in range(5):
            rows+=1
            row_name.append([type_name[typei]+"_"+cell_name[celli]+"_KFold_"+str(i)])

            testData = np.array(pd.DataFrame([test_1.iloc[:,i],test_2.iloc[:,i],test_3.iloc[:,i],test_4.iloc[:,i],test_Blastn]).T)

            newTestData_one = np.array([])
            newTestLabel_one = np.array([])

            kf = KFold(5,True,10)
            Sen = 0
            Spe = 0
            Acc = 0
            Mcc = 0
            Auc = 0
            Th = 0
            for j,[train_index, test_index] in enumerate(kf.split(testData)):

                X_train = testData[train_index]
                X_test = testData[test_index]
                Y_train = testLabel[train_index]
                Y_test = testLabel[test_index]

                model = LogisticRegression()
                clf =model.fit(X_train, Y_train)
                th,_,_,_,_,_,_,acc,mcc,auc = metricsCal.get_train_metrics(clf.decision_function(X_train).reshape(-1),Y_train.reshape(-1))
                _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics(clf.decision_function(X_test).reshape(-1),Y_test.reshape(-1),th)
                newTestData_one = np.append(newTestData_one,clf.decision_function(X_test).reshape(-1),axis = 0)
                newTestLabel_one = np.append(newTestLabel_one,Y_test.reshape(-1),axis = 0)

                Sen+=sen
                Spe+=spe
                Acc+=acc
                Mcc+=mcc
                Auc+=auc
                Th+=th
                print(auc)
            if i == 0:
                newTestLabel = np.append(newTestLabel,newTestLabel_one,axis=0)
                newTestData = np.append(newTestData,newTestData_one,axis = 0)
            else:
                newTestData = np.append(newTestData.reshape(-1,i),newTestData_one.reshape(-1,1),axis = 1)

            Param[rows][0] = Sen/5
            Param[rows][1] = Spe/5
            Param[rows][2] = Acc/5
            Param[rows][3] = Mcc/5
            Param[rows][4] = Auc/5
            Param[rows][5] = Th/5
        pd.DataFrame(np.append(newTestLabel.reshape(-1,1),newTestData,axis=1)).to_csv("Result/Stacking_Fusion_With_Blastn/"+type_name[typei]+"_"+cell_name[celli]+"testFold.csv",header = ["true","KFold_0","KFold_1","KFold_2","KFold_3","KFold_4"])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8671857552049935
0.8661851290476588
0.8579759211182683
0.8699273410199967
0.8700882796047636


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.8618024881364627
0.8639985109622454
0.8608261365991083
0.8661194152388869
0.8724577622149882


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


...

0.7490780319458163
0.7599970770598954
0.7563981555404671
0.7386139044882678
0.7306678025227197


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7428736817542452
0.7572227715691213
0.7498106668258433
0.7344676828547796
0.724647972714582


/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.740201846452897
0.7577977201067184
0.7534922488428604
0.7289641597791005
0.7280113917242107


In [50]:
pd.DataFrame(np.append(np.array(row_name).reshape(-1,1),Param,axis=1)).to_csv("Table/Stacking_Result.csv",header=header_name,index=0)

In [51]:
np.mean(Param[:,-2])

0.8165320383582648